# Elasticipy for the impatient

This notebook is designed to give a quick overview of the features of Elasticipy, and illustrate basic syntaxes.

## Very simple example: isotropic case

### Stiffness tensor and derived properties

Let's start by creating a simple isotropic stiffness tensor:

In [ ]:
from Elasticipy.tensors.elasticity import StiffnessTensor

In [ ]:
C=StiffnessTensor.isotropic(E=210e3, nu=0.25)
print(C)

Some engineering constants can be directly derived from it:

In [ ]:
G=C.shear_modulus.mean()
K=C.bulk_modulus
print('G={}, K={}'.format(G,K))

The stiffness tensor can actually be constructed from any combinations of $E$, $\nu$, $K$, $\lambda$ or G (or equivalently $\mu$), e.g.:

In [ ]:
print(StiffnessTensor.isotropic(E=210e3,lame2=G))

### Compute stress/strain relationship

Once constructed, the stiffness tensor can be used to establish the stress-strain relationship.

#### Simple tensile strain

Let's start by creating a strain tensor:

In [ ]:
from Elasticipy.tensors.stress_strain import StrainTensor
eps = StrainTensor.tensile([1,0,0],1e-3)
print(eps)

Then apply the generalized Hooke's law:

In [ ]:
sigma = C * eps
print(sigma)

One can compute the equivalent stresses:

In [ ]:
print(sigma.vonMises())
print(sigma.Tresca())

#### Simple tensile stress

Now, we want to define the stress, and compute the relating strain.

In [ ]:
from Elasticipy.tensors.stress_strain import StressTensor
sigma = StressTensor.tensile([1,0,0],100)

We need the compliance tensor first:

In [ ]:
S = C.inv()
print(S)

Then, the generalized Hooke's law gives:

In [ ]:
eps = S * sigma
print(eps)

One can estimate the corresponding volumetric elastic energy density:

In [ ]:
print(eps.elastic_energy(sigma))

## Array of strains/stresses

In order to simplify the syntax and speed up calculations, Elasticipy introduces the concept of *tensor array*. As an example, we create a combination of constant tensile strain (along $Z$) and a shear stress in $(X-Y)$ plane with increasing magnitude:

In [ ]:
tau = range(100)
sigma = StressTensor.shear([1,0,0],[0,1,0],tau) + StressTensor.tensile([1,0,0],100)
print(sigma)

We can see the the resulting array is of shape (100,). Let's have a look of its end values:

In [ ]:
print(sigma[0]) # Chek out the usual syntax when dealing with array!
print(sigma[-1])

Most Elasticipy's commands can be broadcasted. E.g.:

In [ ]:
print(sigma.vonMises())

Conversely, the corresponding *strain array* can be computed at once:

In [ ]:
eps = S * sigma
print(eps)

As ``sigma`` is an array of shape (100,), so is ``eps``.

In [ ]:
print(eps[0])
print(eps[-1])

## Mohr circles

Given a stress tensor, one can easily plot it through the well-known Mohr circles. E.g.:

In [ ]:
fig,_= sigma[-1].draw_Mohr_circles() # Mohr circles drawn from the last stress value

## Anisotropic elasticity

We now consider the (slightly) more complex case of anisotropic elasticity. We use monoclinic TiNi as an example:

In [41]:
C = StiffnessTensor.monoclinic(phase_name='TiNi',
                                C11=231, C12=127, C13=104,
                                C22=240, C23=131, C33=175,
                                C44=81, C55=11, C66=85,
                                C15=-18, C25=1, C35=-3, C46=3)
print(C)

Stiffness tensor (in Voigt mapping):
[[231. 127. 104.   0. -18.   0.]
 [127. 240. 131.   0.   1.   0.]
 [104. 131. 175.   0.  -3.   0.]
 [  0.   0.   0.  81.   0.   3.]
 [-18.   1.  -3.   0.  11.   0.]
 [  0.   0.   0.   3.   0.  85.]]
Phase: TiNi
Symmetry: monoclinic


One can check *how much* this material is anisotropic by checking out its Universal Anisotropy index:

In [43]:
print(C.universal_anisotropy)

5.141009551641412


Because of this anisotropy, the Young modulus is not constant over every direction:

In [44]:
E = C.Young_modulus
print(E)

Spherical function
Min=26.283577707639257, Max=191.39659146987594


This means that E ranges in 26 to 191 GPa, depending on the tensile direction. For instance, its value measured along $X$, $Y$ and $Z$ are:

In [45]:
print(E.eval([[1,0,0],[0,1,0],[0,0,1]]))

[124.5223244  120.92120855  96.13750722]


### Plotting engineering elastic constants

The spatial dependence of the Young modulus can illustrated by different ways:

#### As a 3D surface

In [46]:
E.plot3D()

(<Figure size 640x480 with 2 Axes>,
 <Axes3D: xlabel='X', ylabel='Y', zlabel='Z'>)

#### As planar sections

In [47]:
E.plot_xyz_sections()

(<Figure size 640x480 with 3 Axes>,
 [<PolarAxes: title={'center': 'X-Y plane'}>,
  <PolarAxes: title={'center': 'X-Z plane'}>,
  <PolarAxes: title={'center': 'Y-Z plane'}>])

#### As a a pole figure

In [51]:
E.plot_as_pole_figure() # Lambert projection (by default)

(<Figure size 640x480 with 2 Axes>, <PolarAxes: >)

#### Hyperspherical functions

This spatial dependence applies for other engineering moduli, like the Poisson ratio ($\nu$), except that $\nu$ depends on 2 orthogonal directions; hence the concept of *hyperspherical functions*:

In [ ]:
nu=C.Poisson_ratio
print(nu)

In this case, for a given direction, one can plot the mean value for every other orthogonal directions:

In [ ]:
nu.plot_xyz_sections()

Similarly, for a given direction, one can check out the min/max values for every orthogonal directions:

In [ ]:
nu.plot_as_pole_figure(which='min')
nu.plot_as_pole_figure(which='max')

## Rotations of tensors

Any stiffness or compliance tensor can be rotated by taking advantage of ``scipy.spatial.transform``:

### Apply a single rotation

Let's assume that we want to rotate the stiffness tensor by 45° aroung the $X$ direction:

In [ ]:
from scipy.spatial.transform import Rotation
rotation = Rotation.from_euler('X', 45, degrees=True)

The rotation of $C$ is simply made by multiplying it by the ``rotation`` object:

In [ ]:
C_rotated = C * rotation
print(C_rotated)

One can check that the engineering properties have been rotated accordingly:

In [ ]:
C_rotated.Young_modulus.plot3D()

### Apply multiple rotations

As for stress and strain, one can perform multiple rotations at once, leading to an *array of stiffnesses*:

In [ ]:
rotations = Rotation.random(100)
C_rotated = C * rotations
print(C_rotated)

This array can be used to compute the averages, such as the Hill average:

In [ ]:
C_hill = C_rotated.Hill_average()
print(C_hill)

One can check that such aggregate is almost isotropic:

In [ ]:
print(C_hill.universal_anisotropy)

If one wants to consider an infinite set of rotations, he/she can compute the corresponding average from the single stiffness tensor. In this case, the analytic solution lead to:

In [ ]:
print(C.Hill_average())

which is obviously strictly isotropic:

In [ ]:
print(C.Hill_average().universal_anisotropy)
print(C.Hill_average().is_isotropic())

## Wave velocities

The stiffness tensor of a material is highly related to the wave propagation velocities. From a given material and a given direction, one can define three different wave velocities (namely the primary wave, fast secondary and slow secondary wave velocities). They can be computed as follows:

In [ ]:
rho = 6.5 # kg/dm^3 !
v1, v2, v3 = C.wave_velocity(rho)
print(v1)

Note that, in order to keep unit consistency, the mass density is provided in $\text{kg}.\text{dm}^{-3}$, whereas the stiffness tensor is given in GPa. Therefore, the wave velocities are returned in $\text{km}.\text{s}^{-1}$ here.
Let's plot the primary wave velocity as a 3D surface:

In [ ]:
v1.plot3D()

## Graphical User Interface

One convenient way to investigate the influence of symmetries on the engineering constants is to use the GUI:

In [52]:
from Elasticipy.gui import crystal_elastic_plotter
crystal_elastic_plotter()

SystemExit: 0

C:\Users\depriester\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
